In [1]:
#setup
!pip install --user -U nltk
import nltk
from nltk.corpus import gutenberg
from nltk.corpus import reuters
from nltk.corpus import abc
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import ngrams
from nltk.lm import Vocabulary

import re
import random
import numpy as np

from math import log, fsum, exp


nltk.download('punkt')
nltk.download('gutenberg')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Vince\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

# **Language Model**
***
A Language Model (**LM**) is a model that given a sequence of words, or a sentence, is able to compute the probability of the sentence or sequence of words. Formally, an LM is a **probability distribution** over the sequence of words:


\begin{align*}
P(W) = P(w_1,w_2,w_3,...w_n)
\end{align*}

where:
- $W$ is the input sentence.
- $n$ is the number of words in the sentence.
- $w_i$ is the i-th word of the sentence.

One of the related task to LM is to compute the probability that a word has of appearing given a sequence:
\begin{align*}
P(w_n | w_1,w_2,w_3,...w_{n-1})
\end{align*}

### CONSTRUCTION OF TRAINING TRAINING SET AND TEST SET
we construct our corpus sentence, in which we have for each sentence a list of words.
After that we split our list into two parts:


1.   80% into **training set**
2.   20% into **test set**



In [2]:
corpus_sentences_nltk = gutenberg.sents(gutenberg.fileids()[:100])
corpus_sentences = []

#For each sentence we remove the punctuations
for sentence in corpus_sentences_nltk:
  tmp = [] 
  for word in sentence:
    #This regular expression is used for removing punctuation
    word = re.sub(r'[^\w\s]', '', word)
    if (word != ''):
      tmp.append(word.lower())
  if (len(tmp)!=0): 
    corpus_sentences.append(tmp) 

random.shuffle(corpus_sentences)
#We split our corpus in training set (80%) and test set (20%)
training_set = corpus_sentences[:int(len(corpus_sentences)*0.80)]
test_set = corpus_sentences[int(len(corpus_sentences)*0.80):]


## **How work Language Model**
***
The construction of an LM is based on **n-grams**
### **N-Gram**
An N-gram is a sequence of N words, for example given n = 2 and a corpus like : 

"I study NLP"

the bigrams is: {I study, study NLP}.

So, given a word $w_n$ and a sequence ranging from $w_1$ to $w_{n-1}$, we can compute the probability that $w_n$ has of appearing immediately after the sequence from $w_1$ to $w_{n-1}$, and this is the **N-gram probability**.


Formally, in general, the **N-gram probability**, given a word $w_n$ and a sequence ranging from $w_1$ to $w_{n-1}$, is a **fraction** where at the denominator we have **the number of occurrences of the n-1-gram** and at the numerator **the number of occurrences of the n-gram**:

\begin{align*}
P(w_n | w_1^{N-1}) = \frac{C(w_1^{N-1}w_{N})}{C(w_1^{N-1})}
\end{align*}

**with**:
\begin{align*}
C(w_1^{N-1}w_N) = C(w_1^N)
\end{align*}

**where**
- $C(w_1^{N-1}w_{N})$ is the number of occurrences of the n-gram.
- $C(w_1^{N-1})$ is the number of occurrences of the n-1-gram.

### **Sequence probabilities**
In order to compute the probability of an entire sequence, is necessary to apply the **Chain Rule**, which is the **joint N-gram probability** of each word of the sequence:

\begin{align*}
P(w_1w_2...w_n) = \prod_{i=1}^{N}P(w_1|w_1w_2...w_{i-1})
\end{align*}

Since for very large *N* there is the risk that the probability is 0, thus canceling the whole product, it is used the **approximation of sequence probability** where we fixed tha value of *N*. 

For example, if we fixed *N=2* in the calculation of the joint probability don't consider the entire previous sequence but only the previous word:
\begin{align*}
P(w_1w_2...w_n) ≈ \prod_{i=1}^{N}P(w_i|w_{i-1})
\end{align*}


In order to compute the probability of the sequence it is necessary:
- Add **n-1 start of sentence token as $<s>$** at the beginning of the sentence, this is done in order to compute the n-gram probability of the initial word.
- Add 1 **end of sentence token as $</s>$** for two reasons:
  - differentiate the same word when it appears at the end of the sentence versus when it appears anywhere else.
  - make sure that the sum of the probabilities of all sentences of any length is 1.


### **UNKNOWN WORDS**
When using Language Model in real tasks, it happens to deal with words that did not occurr in the training set. We need to handle word missing from a training corpus.
We need to:


*   Update training corpus with *\<UNK>*
*   Construct a vocabulary

A way to train the LM for unknow words is to create a vocabulary implicitly, and replace in the training corpus words with frequency is less than 2.



In this code we construct a dictionary in which we saved for each word the frequency.

In [3]:
dict_word_freq = {}
#For each word we compute the frequencies in all the training set
for sentence in training_set:
  for word in sentence:
    dict_word_freq[word] = dict_word_freq.get(word,0) + 1


We replace words in training corpus with frequency < 10 with \<UNK>

In [4]:
#For each sentence in training set we replace words with freq < 10 with <UNK>
for sentence in training_set:
  for index_word,word in enumerate(sentence):
    if (dict_word_freq[word]<10):
      sentence[index_word] = "<UNK>"

We define a function where give in input the training corpus return the vocabulary. 

In [5]:
#We define a funcion where we create the vocabulary
def get_vocabulary(training_set):
  vocabulary_list = []
  for sentence in training_set:
    for word in sentence:
      if(word not in vocabulary_list and word != "<UNK>" and word!= "<e>"):
        vocabulary_list.append(word)
  return vocabulary_list

### **CONSTRUCTION OF N-GRAM LANGUAGE MODEL**
For the construction of N-gram Language Model we need:


*   Count Matrix
*   Probability Matrix with log probability to avoid underflow

Recall that we are considering N-grams approximation, where we fixed the N and we compute for each N-gram in our training corpus the conditional probabilities, for avoiding problem of 0 we add-k smoothing:
\begin{align*}
P(w_n | w_{n-N+1}^{N-1}) = \frac{C(w_{n-N+1}^{N-1},w_{N})+k}{C(w_{n-N+1}^{N-1})+|V|*k}
\end{align*}






### **COUNT MATRIX**
In this section of code we define a function where in input we have:


*   *N* of N-gram
*   *training corpus* 

And return two dictionary where for each N-gram we store the numerator and denominator:


1.   the first ***count_dictionary*** contains the entries of the count matrix, in which we store the frequencies of each N-gram in our training set
2.   the second ***rowsum*** is used for row sum, where we store the sum of the frequencies of each (N-1)-gram of the N-grams in our training set

In [6]:
#In this function we create the count matrix, for avoiding the problem where most entries are 0 we use a dictionary.

'''
  We define two dictionary:
    - the first "count_dictionary" contains the entries of the count matrix, in which we store the frequencies of each N-gram in our training set
    - the second "rowsum" is used for row sum, where we store the sum of the frequencies of each (N-1) previous words of the N-grams in our training set
'''
def get_count_matrix(n,training_set):
  count_dictionary = {}
  rowsum = {}
  ngrams_corpus = []
  for sentence in training_set:
    #For each sentence we add <e> in the final position, this because the function ngram of nltk add N ending symbol, but we work only with one
    if(sentence[len(sentence)-1] != "<e>"):
      sentence.append("<e>")
    ngrams_corpus = (list(ngrams(sentence, n, pad_left=True, left_pad_symbol='<s>')))
    for ngram in ngrams_corpus:
      key_row = ngram[0:n-1]
      count_dictionary[ngram]= count_dictionary.get(ngram,0) + 1
      rowsum[key_row] = rowsum.get(key_row,0) + 1
  return count_dictionary,rowsum
        

### **PROBABILITY MATRIX**
Now we define a function where the input is formed by:


*   ***count_dictionary*** that represents the numerator without the k-smoothing
*   ***rowsum*** that represents the denominator without the k-smoothing
*   ***n*** of N-gram
*   ***V*** is the size of the vocabulary

In this function we compute the ratio of the previous formula.

In [7]:
#In this function we create the probability matrix
def get_probability_matrix(count_dictionary,rowsum,n, V):
  probability_dictionary = {}
  k_smoothing = 0.05
  #For each value of each N-gram we divide it with the corresponding row sum where the row corresponds to the (N-1)-gram of the N-gram.
  for key,value in count_dictionary.items():
    key_row = key[0:n-1] #In this key_row we store the key for the dictionary rowsum, that is equal to the (N-1)-gram of the N-gram
    probability_dictionary[key] = log((count_dictionary[key] + k_smoothing) / (rowsum.get(key_row,0) + (V * k_smoothing)),2)
  return probability_dictionary

    

From a training corpus we can construct:


*   Vocabulary
*   Count matrix in the code is ***count_dictionary*** and ***rowsum***
*   Probability matrix in the code is represented by ***probability_dictionary***


In [8]:
N = 3 #We define N of the N-gram approximation
vocabulary_list = get_vocabulary(training_set)

count_dictionary, rowsum= get_count_matrix(N,training_set)

probability_dictionary = get_probability_matrix(count_dictionary, rowsum,N, len(vocabulary_list))

#ONLY FOR THE PRINT
for key, value in list(probability_dictionary.items())[10:20]:
  print("[COUNT =", key, count_dictionary[key],"] [ROWSUM =",key[0:N-1], rowsum[key[0:N-1]], "] [PROBABILITY =", pow(2,value),"]")


[COUNT = ('thyself', 'accompanied', '<UNK>') 1 ] [ROWSUM = ('thyself', 'accompanied') 1 ] [PROBABILITY = 0.002370470707754824 ]
[COUNT = ('accompanied', '<UNK>', 'not') 1 ] [ROWSUM = ('accompanied', '<UNK>') 1 ] [PROBABILITY = 0.002370470707754824 ]
[COUNT = ('<UNK>', 'not', 'social') 1 ] [ROWSUM = ('<UNK>', 'not') 312 ] [PROBABILITY = 0.001392665296107169 ]
[COUNT = ('not', 'social', 'communication') 1 ] [ROWSUM = ('not', 'social') 1 ] [PROBABILITY = 0.002370470707754824 ]
[COUNT = ('social', 'communication', 'yet') 1 ] [ROWSUM = ('social', 'communication') 1 ] [PROBABILITY = 0.002370470707754824 ]
[COUNT = ('communication', 'yet', 'so') 1 ] [ROWSUM = ('communication', 'yet') 1 ] [PROBABILITY = 0.002370470707754824 ]
[COUNT = ('yet', 'so', 'pleased') 1 ] [ROWSUM = ('yet', 'so') 17 ] [PROBABILITY = 0.0022878309184006964 ]
[COUNT = ('so', 'pleased', 'canst') 1 ] [ROWSUM = ('so', 'pleased') 9 ] [PROBABILITY = 0.0023284177846767933 ]
[COUNT = ('pleased', 'canst', 'raise') 1 ] [ROWSUM = ('

### **BUILDING TEST SET**
In this function we process our test set in which for each sentece we replace the words that are not in vocabulary with the tag \<UNK> and add the ending symbol *\<e>* 

In [9]:
#In this function we process the test set:
def process_test_set(test_set, vocabulary_list):
  for sentence in test_set:
    for index, word in enumerate(sentence):
      #We replace the words that are no present in the dictionary with <UNK>
      if (word not in vocabulary_list and word != "<e>"):
        sentence[index] = "<UNK>"
    #We add the ending symbol "<e>", this help use for the computation of perplexity!
    if(sentence[len(sentence)-1] != "<e>"):
      sentence.append("<e>")
  return test_set

### **PERPLEXITY - A MEASURE OF QUALITY OF LM**
Given a sentence:
\begin{align*}
W = w_1, w_2, ... , w_m
\end{align*}
and its probability:
\begin{align*}
P(W) = P(w_1, w_2, ... , w_m)
\end{align*}
If it is higher, the more accurate (or even realistic) the sentence ***W*** is.

Given a laguage mode, we do not use directly the probability of a test sentence to evaluate the LM effectiveness, rather we use a measure called **PERPLEXITY** defined as:

\begin{align*}
PP(W) = P(w_1, w_2, ... , w_m)^{-1/m} = \sqrt[m]{\frac{1}{P(w_1,w_2, ..., w_m)}} = \sqrt[N]{\prod_{i=1}^{m}\frac{1}{P(w_i|w_1, w_2, ..., w_{i-1})}}
\end{align*}


if we use a bigram LM, the the perplexity become:
\begin{align*}
\sqrt[m]{\prod_{i=1}^{m}\frac{1}{P(w_i|w_{i-1})}}
\end{align*}

The LM can be evaluated on test sets using the **PERPLEXITY METRIC**
\begin{align*}
PP(W) = P(s_1, s_2, ... , s_N)^{-1/m}
\end{align*}

where:


*   ***W*** -> test set containing ***N*** sentences ${s_j}$, $j=1, ..., N$
*   ${s_j}$ -> j-th sentence in the test set, each ending with ***\</s>***
*   ***m*** -> number of all words in the entire test set ***W*** including ***\</s>*** but not including ***\<s>***.

For erase of computation, the log peplexity is often used:
\begin{align*}
\log_2 PP(W) = - \frac{1}{m} * \sum_{i=1}^{m}\log_2 (P(w_i|w_{i-1}))
\end{align*}

\begin{align*}
PPL = 2^{\log_2 PP(W)}
\end{align*}




In [10]:
ngrams_ = []
probability = []
total_words = 0
k_smoothing = 0.05
test_set_processed = process_test_set(test_set, vocabulary_list)
for sentence in test_set_processed:
    _sum = 0
    total_words = total_words + len(sentence) #We sum the length of each sentence for the computation of perplexity
    ngrams_ =  (list(ngrams(sentence, N, pad_left=True, left_pad_symbol='<s>'))) #We compute the N-grams for each sentence of the test set
    for ngram in ngrams_:
      #We check if N-gram is contained in probability_dictionary we pick that value
      if (ngram in probability_dictionary): 
        _sum = _sum + probability_dictionary[ngram]
      else:
      #Otherwise we estimate the probability using the k-smoothing
        key_row = ngram[0:N-1]
        tmp = (count_dictionary.get(ngram,0) + k_smoothing) / (rowsum.get(key_row,0) + (len(vocabulary_list) * k_smoothing))
        _sum = _sum + log(tmp,2)
    probability.append(_sum)
 
#We compute the log_perplexity
log_perplexity = (-1/total_words) * (fsum(probability))

#We do the inverse of logarithm in order to have the effective value of perplexity
perplexity = pow(2,log_perplexity)
print("PERPLEXITY: ",perplexity)


PERPLEXITY:  954.2482469286284


In this function we compute upcoming word given the previus N-1 gram, and return a dictionary where for each word we have the probability

In [11]:
#We define a function that suggest the upcoming words with their probability
def get_upcoming_words(last_ngram, probability_dictionary):
  probability_list = {}

#We consider all tha N-gram that contain the (N-1)-gram. (N-1)-gram is the last in our phrase
  for key,value in probability_dictionary.items():
    key_row = (key[0:N-1])
    if(key_row == last_ngram):
      probability_list[key[N-1]] = value
  probability_list.pop("<UNK>",None)
  sorted_prob= dict(sorted(probability_list.items(), key=lambda x:x[1],reverse=True))
  return sorted_prob

Given a phrase we compute the upcoming words and print the 5 most probable words

In [13]:
#Our phrase
phrase = "i have just"

#We tokenize our phrase
phrase_tokenize = [word.lower() for word in word_tokenize(phrase)]

#We consider only the lase N-1 gram
last_ngram = tuple(phrase_tokenize[len(phrase_tokenize)-N+1:len(phrase_tokenize)])

#Compute the upcoming word
upcoming_words = get_upcoming_words(last_ngram,probability_dictionary)
for key, value in list(upcoming_words.items())[:10]:
  print(phrase, key, " -> Probability =", pow(2,value))

i have just been  -> Probability = 0.008729388942773999
i have just such  -> Probability = 0.002263174911089555
i have just taken  -> Probability = 0.002263174911089555
i have just now  -> Probability = 0.002263174911089555
i have just freed  -> Probability = 0.002263174911089555
i have just felt  -> Probability = 0.002263174911089555
i have just ask  -> Probability = 0.002263174911089555
i have just prevailed  -> Probability = 0.002263174911089555
i have just come  -> Probability = 0.002263174911089555
i have just balances  -> Probability = 0.002263174911089555
